In [2]:
from keras.models import load_model

model = load_model('model1.h5')

Using TensorFlow backend.


In [10]:
import os
import PIL
from PIL import Image
from keras.applications.vgg16 import preprocess_input, VGG16

DATA_FOLDER = 'data/test_stg2/'
PP_TEST_DATA = 'data/pp_test2_data.npy'
SHAPE = (300, 500, 3)

files = os.listdir(DATA_FOLDER)
n_files = len(files)

In [11]:
import numpy as np

batch_x = np.ndarray((n_files, *SHAPE), dtype=np.float32)

for idx, file in enumerate(files):
    print('%04d/%04d' % (idx, n_files), end='\r')
    path = os.path.join(DATA_FOLDER, file)
    img = Image.open(path)
    img = img.resize(SHAPE[:2][::-1])
    img = np.asarray(img, dtype=np.float32)
    batch_x[idx] = img

In [12]:
batch_x = preprocess_input(batch_x)

In [13]:
base_model = VGG16(include_top=False, input_shape=SHAPE)
base_model.layers.pop()
preds = base_model.predict(batch_x, batch_size=16, verbose=1)

12153/12153 [==============================] - 1529s  


In [15]:
np.save(PP_TEST_DATA, preds)

In [16]:
preds = np.load(PP_TEST_DATA)

In [17]:
preds.shape

(12153, 9, 15, 512)

In [18]:
result = model.predict(preds, verbose=1)

12153/12153 [==============================] - 17s    


In [21]:
import pandas as pd

METADATA = 'metadata.npy'
metadata = np.load(METADATA).item()
cols = ['image','ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT']
df = pd.DataFrame(columns=cols)

In [25]:
df['image'] = ['test_stg2/' + f for f in files]
df.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_11459.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,test_stg2/image_10781.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,test_stg2/image_06154.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,test_stg2/image_11821.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,test_stg2/image_11635.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
for col in cols[1:]:
    df[col] = result[:, np.argmax(metadata[col])]

df.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_11459.jpg,0.924618,0.004355,0.002970,0.001399,0.052963,0.001906,0.002618,0.009172
1,test_stg2/image_10781.jpg,0.402149,0.082876,0.007072,0.005011,0.388063,0.038563,0.017514,0.058751
2,test_stg2/image_06154.jpg,0.818096,0.017827,0.003842,0.013560,0.048237,0.003596,0.033409,0.061433
3,test_stg2/image_11821.jpg,0.818837,0.011370,0.004948,0.002379,0.128117,0.007060,0.007577,0.019712
4,test_stg2/image_11635.jpg,0.815055,0.028674,0.003532,0.004618,0.102342,0.007310,0.004337,0.034132


In [28]:
df.to_csv('data/submission_test2.csv', index=False)

# Merge submissions

In [29]:
df2 = pd.read_csv('data/submission_test1.csv')

In [34]:
df2.sort_values('image', inplace=True)

In [36]:
df2.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
98,img_00005.jpg,0.002081,0.000037,0.000018,0.000007,0.997741,0.000070,0.000020,0.000026
709,img_00007.jpg,0.841953,0.010561,0.002981,0.006471,0.003779,0.024229,0.003413,0.106613
141,img_00009.jpg,0.998159,0.000564,0.000054,0.000041,0.000054,0.000201,0.000043,0.000884
124,img_00018.jpg,0.953298,0.008528,0.001035,0.002662,0.001320,0.008410,0.003950,0.020796
669,img_00027.jpg,0.194681,0.013352,0.001617,0.001859,0.005537,0.751229,0.006689,0.025035


In [38]:
df.sort_values('image', inplace=True)

In [39]:
df.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
8874,test_stg2/image_00001.jpg,0.419970,0.038963,0.006004,0.003762,0.388857,0.011125,0.075457,0.055862
5563,test_stg2/image_00002.jpg,0.978235,0.007304,0.001142,0.000593,0.006650,0.001080,0.000795,0.004202
7759,test_stg2/image_00003.jpg,0.694308,0.008904,0.001677,0.002752,0.272072,0.003790,0.003122,0.013374
10507,test_stg2/image_00004.jpg,0.979563,0.000376,0.000949,0.000379,0.002044,0.000510,0.000217,0.015962
11103,test_stg2/image_00005.jpg,0.832575,0.040178,0.019336,0.010435,0.012434,0.020140,0.030905,0.033998


In [47]:
dfout = df2.append(df)

In [49]:
dfout.to_csv('data/submission_final.csv', index=False)